In [1]:
from sqlalchemy.types import INT as sqlINT, FLOAT as sqlFLOAT, VARCHAR as sqlVARCHAR
from sqlalchemy import create_engine
from urllib.request import urlopen
from unicodedata import normalize
from io import StringIO
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

Read the database connection details from the repository's `secrets` and create the `engine` that will be used to connect to the database.

In [2]:
host = os.getenv("DATABASE_HOST")
user = os.getenv("DATABASE_USERNAME")
passwd = os.getenv("DATABASE_PASSWORD")
db = os.getenv("DATABASE")

engine = create_engine(
    f'mysql+mysqlconnector://{user}:{passwd}@{host}/{db}',
    echo=False,
    connect_args={'ssl_ca': '/etc/ssl/certs/ca-certificates.crt'}
    )

In [3]:
url = 'https://www.cdc.gov/brfss/annual_data/2019/pdf/codebook19_llcp-v2-508.HTML'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
html_tables = soup.findAll('table',{'class': 'table'})

Transform all the tables found in the codebook url to a pandas dataframe.

_NOTE:_ The attributes the variables are built into a dictionary instead of using a dataframe because a dataframe uses more memory than a dictionary. The reason why the catalog is built into a dataframe is because pandas offers a method to read a table directly from html. 

In [4]:
data_attributes_dict = {}
value_catalogs = pd.DataFrame()

for table in html_tables[1:]:
    # Find the first cell in the table
    attributes_html = table.find('td')
    # Extract the text from the cell
    attributes_text = normalize('NFKD',attributes_html.get_text(separator='\n'))
    # Split the text into lines and create a dictionary that will be used
    #  build a dataframe to store the attributes of the SAS variable
    d = dict()
    for line in attributes_text.splitlines():
        k = str(line.split(':')[0]).strip()
        v = str(line.split(':')[1]).strip()
        d[k] = v
    
    # Skip to the next table if the cell doesn't contain a 'SAS Variable Name', 
    if 'SAS Variable Name' not in d:
        continue
    
    # Extract the SAS Variable Name and remove it from the dictionary
    #  otherwise, it would be used as a column in the resulting dataframe
    code = d.pop('SAS Variable Name')

    # Use the variable name as the key for the dictionary
    data_attributes_dict[code] = d

    # Remove the first row from the table
    table.find('tr').decompose()
    # Remove the colgroup from the table
    table.find('colgroup').decompose()
    
    # Read the table into a dataframe
    df = pd.read_html(StringIO(str(table)))[0]
    # Add the `code` to all the rows in the current iteration of the dataframe
    #  in a new column
    df['Code'] = code
    # Set the index of the dataframe to the `code` and `value` columns
    df.set_index(['Code','Value'],inplace=True)
    # Concatenate the dataframe to the value_catalogs dataframe
    value_catalogs = pd.concat([value_catalogs,df])

#### Attributes

Convert the dictionary of attributes to a dataframe.

In [5]:
attributes = pd.DataFrame(data_attributes_dict).T
attributes

,Label,Section Name,Section Number,Question Number,Column,Type of Variable,Question Prologue,Question,Core Section Number,Module Number
_STATE,State FIPS Code,Record Identification,0,1,1-2,Num,,State FIPS Code,NaN,NaN
FMONTH,File Month,Record Identification,0,8,17-18,Num,,File Month,NaN,NaN
IDATE,Interview Date,Record Identification,0,9,19-26,Char,,Interview Date,NaN,NaN
IMONTH,Interview Month,Record Identification,0,10,19-20,Char,,Interview Month,NaN,NaN
IDAY,Interview Day,Record Identification,0,11,21-22,Char,,Interview Day,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
_FRUITE1,Fruit Exclusion from analyses,Calculated Variables,NaN,17,2153,Num,,Fruit Exclusion from analyses,NaN,12
_VEGETE1,Vegetable Exclusion from analyses,Calculated Variables,NaN,18,2154,Num,,Vegetable Exclusion from analyses,NaN,12
_FLSHOT7,Flu Shot Calculated Variable,Calculated Variables,NaN,1,2155,Num,,Adults aged 65+ who have had a flu shot within...,NaN,13
_PNEUMO3,Pneumonia Vaccination Calculated Variable,Calculated Variables,NaN,2,2156,Num,,Adults aged 65+ who have ever had a pneumonia ...,NaN,13


Change the name of the index column to 'Code'

In [6]:
attributes.index.name = 'Code'

Replace all the spaces in the dataframe column names with underscores.

In [9]:
attributes.columns = [colName.strip().replace(' ','_') for colName in attributes.columns]
attributes.head(3)

,Label,Section_Name,Section_Number,Question_Number,Column,Type_of_Variable,Question_Prologue,Question,Core_Section_Number,Module_Number
Code,,,,,,,,,,
_STATE,State FIPS Code,Record Identification,0,1,1-2,Num,,State FIPS Code,NaN,NaN
FMONTH,File Month,Record Identification,0,8,17-18,Num,,File Month,NaN,NaN
IDATE,Interview Date,Record Identification,0,9,19-26,Char,,Interview Date,NaN,NaN


Create a json file from the dataframe and save it to the `data` folder.

In [ ]:
attributes.to_json('../data/attributes.json',orient='table')

Upload the `attributes` dataframe to the database.

In [17]:
column_types = { 'Code': sqlVARCHAR(10) }
pd.io.sql.to_sql(attributes, 'attributes', engine, if_exists='replace', index=True, flavor='mysql', dtype=column_types)
pd.io.sql.read_sql('SELECT * FROM attributes', engine,index_col='Code')

,Label,Section_Name,Section_Number,Question_Number,Column,Type_of_Variable,Question_Prologue,Question,Core_Section_Number,Module_Number
Code,,,,,,,,,,
_STATE,State FIPS Code,Record Identification,0,1,1-2,Num,,State FIPS Code,None,None
FMONTH,File Month,Record Identification,0,8,17-18,Num,,File Month,None,None
IDATE,Interview Date,Record Identification,0,9,19-26,Char,,Interview Date,None,None
IMONTH,Interview Month,Record Identification,0,10,19-20,Char,,Interview Month,None,None
IDAY,Interview Day,Record Identification,0,11,21-22,Char,,Interview Day,None,None
...,...,...,...,...,...,...,...,...,...,...
_FRUITE1,Fruit Exclusion from analyses,Calculated Variables,None,17,2153,Num,,Fruit Exclusion from analyses,None,12
_VEGETE1,Vegetable Exclusion from analyses,Calculated Variables,None,18,2154,Num,,Vegetable Exclusion from analyses,None,12
_FLSHOT7,Flu Shot Calculated Variable,Calculated Variables,None,1,2155,Num,,Adults aged 65+ who have had a flu shot within...,None,13


#### Catalog

Display the `value_catalogs` dataframe that was built from all the tables in the codebook url.

In [11]:
value_catalogs

Value Label  Frequency  \
Code     Value                                                                 
_STATE   1                                                Alabama       7052   
         2                                                 Alaska       2977   
         4                                                Arizona       8941   
         5                                               Arkansas       5359   
         6                                             California      11613   
...                                                           ...        ...   
_PNEUMO3 BLANK                   Age Less Than 65 Notes: AGE < 65     259156   
_AIDTST4 1                                 Yes Notes: HIVTST7 = 1     126661   
         2                                  No Notes: HIVTST7 = 2     235086   
         9      Don’t know/Not Sure/Refused Notes: HIVTST7 = 7...      16230   
         BLANK      Not asked or missing Notes: HIVTST7 = Missing      40291   

               Percentage Weighted Percentage  
Code     Value                                 
_STATE   1           1.69                1.51  
         2           0.71                0.22  
         4           2.14                2.24  
         5           1.28                0.92  
         6           2.78               12.22  
...                   ...                 ...  
_PNEUMO3 BLANK          .                   .  
_AIDTST4 1          33.51               41.30  
         2          62.20               54.97  
         9           4.29                3.73  
         BLANK          .                   .  

[1942 rows x 4 columns]

In [23]:
value_catalogs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1942 entries, ('_STATE', 1) to ('_AIDTST4', 'BLANK')
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Value Label          1890 non-null   object
 1   Frequency            1942 non-null   int64 
 2   Percentage           1942 non-null   object
 3   Weighted Percentage  1942 non-null   object
dtypes: int64(1), object(3)
memory usage: 88.9+ KB


Modify the values in the `Weighted Percentage` and `Percentage` columns in the dataframe from '.' to NaN and all the other values to float.

In [13]:
#value_catalogs = value_catalogs.apply(pd.to_numeric, errors='ignore') # Alternative way to convert to numeric
value_catalogs.replace({'Percentage':{'.': np.nan}}, inplace=True)
value_catalogs.replace({'Weighted Percentage':{'.': np.nan}}, inplace=True)
value_catalogs = value_catalogs.astype({'Percentage': float, 'Weighted Percentage': float})
print(value_catalogs.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1942 entries, ('_STATE', 1) to ('_AIDTST4', 'BLANK')
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Value Label          1890 non-null   object 
 1   Frequency            1942 non-null   int64  
 2   Percentage           1669 non-null   float64
 3   Weighted Percentage  1669 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 88.9+ KB
None


Find the lenght of the longest string in the `Value` index and use it to set the width of the column types used in the `to_sql` function.

In [32]:
value_len = value_catalogs.index.get_level_values('Value').str.len().max()
value_len

14.0

Create a json file from the dataframe and save it to the `data` folder.

In [ ]:
value_catalogs.to_json('../data/catalog.json',orient='table')

Upload the `value_catalogs` dataframe to the database.

In [34]:
column_types = { 'Code': sqlVARCHAR(10), 'Value': sqlVARCHAR(value_len)}

pd.io.sql.to_sql(value_catalogs, 'catalog', engine, if_exists='replace', index=True, flavor='mysql', dtype=column_types)
pd.io.sql.read_sql('SELECT * FROM catalog', engine, index_col=['Code', 'Value'])

Value Label  Frequency  \
Code     Value                                                                 
_STATE   1                                                Alabama       7052   
         2                                                 Alaska       2977   
         4                                                Arizona       8941   
         5                                               Arkansas       5359   
         6                                             California      11613   
...                                                           ...        ...   
_PNEUMO3 BLANK                   Age Less Than 65 Notes: AGE < 65     259156   
_AIDTST4 1                                 Yes Notes: HIVTST7 = 1     126661   
         2                                  No Notes: HIVTST7 = 2     235086   
         9      Don’t know/Not Sure/Refused Notes: HIVTST7 = 7...      16230   
         BLANK      Not asked or missing Notes: HIVTST7 = Missing      40291   

                Percentage  Weighted Percentage  
Code     Value                                   
_STATE   1            1.69                 1.51  
         2            0.71                 0.22  
         4            2.14                 2.24  
         5            1.28                 0.92  
         6            2.78                12.22  
...                    ...                  ...  
_PNEUMO3 BLANK         NaN                  NaN  
_AIDTST4 1           33.51                41.30  
         2           62.20                54.97  
         9            4.29                 3.73  
         BLANK         NaN                  NaN  

[1942 rows x 4 columns]

#### Examples:

Count how many unique values exist in the `Code` index of the `value_catalogs` dataframe.

In [14]:
value_catalogs.index.get_level_values(0).unique()

Index(['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE',
       'SEQNO', '_PSU', 'CTELENM1',
       ...
       '_VEGESU1', '_FRTLT1A', '_VEGLT1A', '_FRT16A', '_VEG23A', '_FRUITE1',
       '_VEGETE1', '_FLSHOT7', '_PNEUMO3', '_AIDTST4'],
      dtype='object', name='Code', length=342)

Filter the `value_catalog` dataframe to find only rows where, when grouped by Code, the last item in the catalog of responses contributes no more than 30% of the total responses.

This is done because the last item in the catalog of responses is usually reserved for answers like 'Unknown' or 'Not asked, or 'Missing'. If the last item in the catalog of responses is greater than 30%, it is likely that the last item is a 'catch-all' for these types of responses. 

In [15]:
promising_codes = []
for code in value_catalogs.index.get_level_values(0).unique():
    freq = value_catalogs.loc[code]['Frequency']
    if freq.iloc[-1]/freq.sum() < 0.30:
        promising_codes.append(code)

print(len(promising_codes))

131


Example of how to filter the dataframe `value_catalogs` to find the meaning of the responses to the question that has the code `_STATE`.

In [16]:
pd.DataFrame(value_catalogs.loc['_STATE']['Value Label'].head())

,Value Label
Value,
1,Alabama
2,Alaska
4,Arizona
5,Arkansas
6,California


Example of how to filter the dataframe `value_catalogs` to find the meaning of the response `'3'` to the question that has the code `GENHLTH` .

In [17]:
value_catalogs.loc['GENHLTH']['Value Label'].loc['3']

'Good'

Example of how to filter the dataframe `value_catalogs` to find the meaning of all the possible responses to the question that has the code `HLTHPLN1` .

In [18]:
pd.DataFrame(value_catalogs.loc['HLTHPLN1']['Value Label'])

,Value Label
Value,
1,Yes
2,No
7,Don’t know/Not Sure
9,Refused
BLANK,Not asked or Missing
